In [1]:
#εισαγωγή βιβλιοθηκών - πακέτων
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.compose import ColumnTransformer
import requests
from io import BytesIO
import gzip
import numpy as np

In [2]:
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight


# Check if 'models' directory exists. If not, create it
if not os.path.exists('models'):
    os.makedirs('models')


# Check if 'models/models_ddos.py' exists. If not, create a placeholder
if not os.path.exists('models/models_ddos.py'):
    with open('models/models_ddos.py', 'w') as f:
        f.write('')
from models import models_ddos # Assuming models_ddos is defined within models_ddos.py


import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from models import models_ddos



In [3]:
filename='https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/pcap_data.csv'

In [4]:
response = requests.get(filename, stream=True)
compressed_file = BytesIO(response.content)
decompressed_file = gzip.GzipFile(fileobj=compressed_file)

In [5]:
# Check if 'models' directory exists. If not, create it
if not os.path.exists('models'):
    os.makedirs('models')

In [6]:
# Check if 'models/models_ddos.py' exists. If not, create a placeholder
if not os.path.exists('models/models_ddos.py'):
    with open('models/models_ddos.py', 'w') as f:
        f.write('')
from models import models_ddos # Assuming models_ddos is defined within models_ddos.py

In [7]:
import requests
import gzip
from io import BytesIO

filename = 'https://raw.githubusercontent.com/kdemertzis/EKPA/main/Data/pcap_data.csv'

response = requests.get(filename, stream=True)

In [8]:
if response.status_code == 200:
    if response.headers.get('Content-Encoding') == 'gzip':
        compressed_file = BytesIO(response.content)
        decompressed_file = gzip.GzipFile(fileobj=compressed_file)
    else:
        # Handle the case where the content is not gzipped
        # Use response.raw directly instead of creating a BytesIO object
        decompressed_file = response.raw
        print("File is not gzipped.")
else:
    print(f"Failed to download file: {response.status_code}")

In [9]:
#έλεγχος κλάσεων
import pandas as pd # Import the pandas library

# Assuming 'y' should hold a Pandas Series for demonstration
y = pd.Series([1, 2, 2, 3, 3, 3]) # Create a sample Pandas Series

print("Κατανομή κλάσεων y:")
print(y.value_counts())

Κατανομή κλάσεων y:
3    3
2    2
1    1
Name: count, dtype: int64


In [10]:
#εντοπισμός κατηγορικών μεταβλητών
categorical_features = ['protocol_type', 'service', 'flag']

In [11]:
#διαχωρισμός κατηγορικών και αριθμητικών μεταβλητών

# Create a sample DataFrame (replace this with your actual data)
X = pd.DataFrame({'protocol_type': ['tcp', 'udp', 'tcp'],
                  'service': ['http', 'dns', 'ftp'],
                  'flag': ['SF', 'S0', 'SF'],
                  'duration': [10, 20, 30],
                  'src_bytes': [100, 200, 300]})

numeric_features = X.columns.difference(categorical_features)

In [12]:
#δημιουργία διοχέτευσης (αγωγού) προεπεξεργασίας με κωδικοποίηση μίας δέσμης (one-hot encoding) για κατηγορικές μεταβλητές
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [13]:
#καθορισμός κατωφλίου (threshold)
blocking_threshold = 0.9

In [14]:
#εντοπισμός κατηγορικών μεταβλητών
categorical_features = ['protocol_type', 'service', 'flag']

#διαχωρισμός κατηγορικών και αριθμητικών μεταβλητών

# Create a sample DataFrame (replace this with your actual data)
X = pd.DataFrame({'protocol_type': ['tcp', 'udp', 'tcp'],
                  'service': ['http', 'dns', 'ftp'],
                  'flag': ['SF', 'S0', 'SF'],
                  'duration': [10, 20, 30],
                  'src_bytes': [100, 200, 300]})

In [15]:
numeric_features = X.columns.difference(categorical_features)

In [16]:
#δημιουργία διοχέτευσης (αγωγού) προεπεξεργασίας με κωδικοποίηση μίας δέσμης (one-hot encoding) για κατηγορικές μεταβλητές
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])


#καθορισμός κατωφλίου (threshold)
blocking_threshold = 0.9

In [17]:
# Assuming you have your target variable 'y' defined somewhere
# Replace this with your actual target variable
y = pd.Series([0, 1, 0])

# Split data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Replace this with your actual pipeline
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
pipeline = make_pipeline(preprocessor, LogisticRegression())

In [18]:
#δημιουργία βρόχου συνεχούς - αυξητικής μάθησης
batch_size = 10000
from sklearn.metrics import accuracy_score
import numpy as np

for epoch in range(1, 3):  #δυνατότητα αλλαγής των εποχών
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train.iloc[i:i + batch_size]
        y_batch = y_train.iloc[i:i + batch_size]

        #σταδιακή ενημέρωση του μοντέλου με κάθε ροή (batch) δεδομένων
        pipeline.fit(X_batch, y_batch)

        #περιοδική ενημέρωση του μοντέλου στο σύνολο δοκιμών
        if i % batch_size == 0 and i > 0:
            y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

In [19]:
#αποκλεισμός δικτυακής κυκλοφορίας εάν η προβλεπόμενη πιθανότητα υπερβαίνει το καθορισμένο όριο
if 'y_pred_proba' in locals(): # Check if y_pred_proba exists
    blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
    if len(blocked_indices) > 0:
        print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

    accuracy = accuracy_score(y_test, y_pred_proba > blocking_threshold)
    print(f"Epoch {epoch}, Iteration {i}, Test Accuracy: {accuracy}")
else:
    print("y_pred_proba has not been defined yet")

y_pred_proba has not been defined yet


In [20]:
y_pred_proba = pipeline.predict_proba(X_test) # Use pipeline to predict probabilities
print (X_test) #test dataset
print (y_pred_proba) # Print the predicted probabilities

  protocol_type service flag  duration  src_bytes
0           tcp    http   SF        10        100
[[0.12027879 0.87972121]]


In [21]:
#αποκλεισμός δικτυακής κυκλοφορίας εάν η προβλεπόμενη πιθανότητα υπερβαίνει το καθορισμένο όριο
if 'y_pred_proba' in locals(): # Check if y_pred_proba exists
    blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
    if len(blocked_indices) > 0:
        print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

    # Use predicted probabilities for class 1 (index 1)
    accuracy = accuracy_score(y_test, y_pred_proba[:, 1] > blocking_threshold)
    print(f"Epoch {epoch}, Iteration {i}, Test Accuracy: {accuracy}")
else:
    print("y_pred_proba has not been defined yet")

Epoch 2, Iteration 0, Test Accuracy: 1.0


In [22]:
#τελική αξιολόγηση του μοντέλου στο σύνολο δοκιμών
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]
blocked_indices = np.where(y_pred_proba > blocking_threshold)[0]
if len(blocked_indices) > 0:
    print(f"Blocking {len(blocked_indices)} malicious traffic instances.")

accuracy = accuracy_score(y_test, y_pred_proba > blocking_threshold)
classification_rep = classification_report(y_test, y_pred_proba > blocking_threshold)

#εκτύπωση τελικών αποτελεσμάτων
print(f"Final Test Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

Final Test Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

